In [1]:
import os
os.chdir("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project")
print(os.getcwd())

/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project


In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandasql as ps
import glob
from dateutil import parser

import warnings
warnings.filterwarnings('ignore')

In [3]:
#_______________________________________________________
#Loading in daily Air Pressure data for years 2014 - 2018

data_ap = pd.read_csv("data/daily_PRESS_2014.csv")

for i in range(2015, 2019):
    data_ap = pd.concat([data_ap, pd.read_csv("data/daily_PRESS_" + str(i) + ".zip")])
    
data_ap = data_ap[["State Code", "County Code", "Date Local", "Arithmetic Mean"]]
data_ap.rename(columns={"Arithmetic Mean":"Average Air Pressure"}, inplace=True)

#Taking air pressure averages across sensor locations
data_ap = data_ap.groupby(["State Code", "County Code", "Date Local"])["Average Air Pressure"].mean().reset_index()

#_______________________________________________________
#Loading in daily Temperature data for years 2014 - 2018

data_t = pd.read_csv("data/daily_TEMP_2014.csv")

for i in range(2015, 2019):
    data_t = pd.concat([data_t, pd.read_csv("data/daily_TEMP_" + str(i) + ".zip")])
    
data_t = data_t[["State Name", "County Name","State Code","County Code", "Date Local", "Arithmetic Mean"]]
data_t.rename(columns={"Arithmetic Mean":"Average Temperature"}, inplace=True)

#Taking temperature averages across distinct sensor locations
data_t = data_t.groupby(["State Name", "County Name", "Date Local","State Code","County Code"])["Average Temperature"].mean().reset_index()


#_______________________________________________________________
#Loading in daily humidity/dew point data for years 2014 - 2018

data_h = pd.read_csv("data/daily_RH_DP_2014.csv")

for i in range(2015, 2019):
    data_h = pd.concat([data_h, pd.read_csv("data/daily_RH_DP_" + str(i) + ".zip")])

data_h = data_h[["State Code", "County Code", "Date Local", "Arithmetic Mean"]]
data_h.rename(columns={"Arithmetic Mean":"Average Humidity"}, inplace=True)

#Taking humidity averages across distinct sensor locations
data_h = data_h.groupby(["State Code", "County Code", "Date Local"])["Average Humidity"].mean().reset_index()

    
#_______________________________________________________________
#Loading in daily windspeed point data for years 2014 - 2018
data_w = pd.read_csv("data/daily_WIND_2014.csv")

for i in range(2015, 2019):
    data_w = pd.concat([data_w, pd.read_csv("data/daily_WIND_" + str(i) + ".zip")])
    
data_w = data_w[["State Code", "County Code", "Date Local", "Arithmetic Mean"]]
data_w.rename(columns={"Arithmetic Mean":"Average Windspeed"}, inplace=True)

#Taking Windspeed averages across distinct sensor locations
data_w = data_w.groupby(["State Code", "County Code", "Date Local"])["Average Windspeed"].mean().reset_index()

In [4]:
data_t.rename({'State Name':'State', 'County Name': 'County'}, axis=1, inplace=True)
data_t['State'] = data_t['State'].str.lower()
data_t['County'] = data_t['County'].str.lower()
data_t['Date Local'] = pd.to_datetime(data_t['Date Local'])
data_t['Year'] = pd.DatetimeIndex(data_t['Date Local']).year
data_t['Month'] = pd.DatetimeIndex(data_t['Date Local']).month
data_t['Day'] = pd.DatetimeIndex(data_t['Date Local']).day
data_t.head()

data_t2 = data_t.groupby(['State', 'County','Year','Month','State Code','County Code'])['Average Temperature'].mean().reset_index()
data_t2.head()
#data_w = data_w.groupby(["State Code", "County Code", "Date Local"])["Average Windspeed"].mean().reset_index()

,State,County,Year,Month,State Code,County Code,Average Temperature
0,alabama,jefferson,2014,1,1,73,37.601300
1,alabama,jefferson,2014,2,1,73,46.520285
2,alabama,jefferson,2014,3,1,73,53.247984
3,alabama,jefferson,2014,4,1,73,63.699870
4,alabama,jefferson,2014,5,1,73,72.347177


In [5]:
#data_t.to_csv("./cleaned_data/EPA_temp_all-years.csv", index=None, header=True)

In [6]:
query = '''SELECT * FROM data_t WHERE Year = 2016'''
data_t_2016 = ps.sqldf(query=query)
#data_t_2016.to_csv("./cleaned_data/EPA_temp_2016.csv", index=None, header=True)

In [7]:
data_ap['Date Local'] = pd.to_datetime(data_ap['Date Local'])
data_ap['Year'] = pd.DatetimeIndex(data_ap['Date Local']).year
data_ap['Month'] = pd.DatetimeIndex(data_ap['Date Local']).month
data_ap['Day'] = pd.DatetimeIndex(data_ap['Date Local']).day
data_ap.head()

,State Code,County Code,Date Local,Average Air Pressure,Year,Month,Day
0,1,73,2014-01-01,999.822917,2014,1,1
1,1,73,2014-01-02,994.027083,2014,1,2
2,1,73,2014-01-03,1004.125000,2014,1,3
3,1,73,2014-01-04,1000.154167,2014,1,4
4,1,73,2014-01-05,991.166667,2014,1,5


In [15]:
query = '''SELECT * FROM data_ap WHERE Year = 2016'''
data_ap_2016 = ps.sqldf(query=query)
data_ap_2016.head()

,State Code,County Code,Date Local,Average Air Pressure,Year,Month,Day
0,1,73,2016-01-01 00:00:00.000000,1002.581250,2016,1,1
1,1,73,2016-01-02 00:00:00.000000,1001.658334,2016,1,2
2,1,73,2016-01-03 00:00:00.000000,996.637500,2016,1,3
3,1,73,2016-01-04 00:00:00.000000,1000.566667,2016,1,4
4,1,73,2016-01-05 00:00:00.000000,1006.175000,2016,1,5


In [20]:
query = '''SELECT DISTINCT [County Code]FROM data_ap_2016'''
check = ps.sqldf(query=query)
print(len(check),'counties with air pressure')

81 counties with air pressure


In [9]:
data_t_ap_2016 = pd.merge(data_t_2016, data_ap_2016, on = ['Year', 'Month', 'Day', 'State Code', 'County Code','Date Local'], how='left')
data_t_ap_2016.head()

,State,County,Date Local,State Code,County Code,Average Temperature,Year,Month,Day,Average Air Pressure
0,alabama,jefferson,2016-01-01 00:00:00.000000,1,73,38.925000,2016,1,1,1002.581250
1,alabama,jefferson,2016-01-02 00:00:00.000000,1,73,38.339584,2016,1,2,1001.658334
2,alabama,jefferson,2016-01-03 00:00:00.000000,1,73,40.785416,2016,1,3,996.637500
3,alabama,jefferson,2016-01-04 00:00:00.000000,1,73,36.147917,2016,1,4,1000.566667
4,alabama,jefferson,2016-01-05 00:00:00.000000,1,73,37.666667,2016,1,5,1006.175000


In [10]:
data_h['Date Local'] = pd.to_datetime(data_h['Date Local'])
data_h['Year'] = pd.DatetimeIndex(data_h['Date Local']).year
data_h['Month'] = pd.DatetimeIndex(data_h['Date Local']).month
data_h['Day'] = pd.DatetimeIndex(data_h['Date Local']).day
data_h.head()

query = '''SELECT * FROM data_h WHERE Year = 2016'''
data_h_2016 = ps.sqldf(query=query)
data_h_2016.head()

,State Code,County Code,Date Local,Average Humidity,Year,Month,Day
0,1,73,2016-01-01 00:00:00.000000,71.083333,2016,1,1
1,1,73,2016-01-02 00:00:00.000000,67.083334,2016,1,2
2,1,73,2016-01-03 00:00:00.000000,67.020834,2016,1,3
3,1,73,2016-01-04 00:00:00.000000,62.520834,2016,1,4
4,1,73,2016-01-05 00:00:00.000000,41.687500,2016,1,5


In [11]:
data_t_ap_h_2016 = pd.merge(data_t_ap_2016, data_h_2016, on = ['Year', 'Month', 'Day', 'State Code', 'County Code','Date Local'], how='left')
data_t_ap_h_2016.head()

,State,County,Date Local,State Code,County Code,Average Temperature,Year,Month,Day,Average Air Pressure,Average Humidity
0,alabama,jefferson,2016-01-01 00:00:00.000000,1,73,38.925000,2016,1,1,1002.581250,71.083333
1,alabama,jefferson,2016-01-02 00:00:00.000000,1,73,38.339584,2016,1,2,1001.658334,67.083334
2,alabama,jefferson,2016-01-03 00:00:00.000000,1,73,40.785416,2016,1,3,996.637500,67.020834
3,alabama,jefferson,2016-01-04 00:00:00.000000,1,73,36.147917,2016,1,4,1000.566667,62.520834
4,alabama,jefferson,2016-01-05 00:00:00.000000,1,73,37.666667,2016,1,5,1006.175000,41.687500


In [12]:
data_w['Date Local'] = pd.to_datetime(data_w['Date Local'])
data_w['Year'] = pd.DatetimeIndex(data_w['Date Local']).year
data_w['Month'] = pd.DatetimeIndex(data_w['Date Local']).month
data_w['Day'] = pd.DatetimeIndex(data_w['Date Local']).day
display(data_w.head())

query = '''SELECT * FROM data_w WHERE Year = 2016'''
data_w_2016 = ps.sqldf(query=query)
data_w_2016.head()

,State Code,County Code,Date Local,Average Windspeed,Year,Month,Day
0,1,73,2014-01-01,45.147917,2014,1,1
1,1,73,2014-01-02,106.842708,2014,1,2
2,1,73,2014-01-03,88.738541,2014,1,3
3,1,73,2014-01-04,52.647916,2014,1,4
4,1,73,2014-01-05,64.483333,2014,1,5


,State Code,County Code,Date Local,Average Windspeed,Year,Month,Day
0,1,73,2016-01-01 00:00:00.000000,145.589584,2016,1,1
1,1,73,2016-01-02 00:00:00.000000,124.310417,2016,1,2
2,1,73,2016-01-03 00:00:00.000000,120.292708,2016,1,3
3,1,73,2016-01-04 00:00:00.000000,84.986458,2016,1,4
4,1,73,2016-01-05 00:00:00.000000,30.745833,2016,1,5


In [13]:
data_t_ap_h_w_2016 = pd.merge(data_t_ap_h_2016, data_w_2016, on = ['Year', 'Month', 'Day', 'State Code', 'County Code','Date Local'], how='left')
data_t_ap_h_w_2016.head()

,State,County,Date Local,State Code,County Code,Average Temperature,Year,Month,Day,Average Air Pressure,Average Humidity,Average Windspeed
0,alabama,jefferson,2016-01-01 00:00:00.000000,1,73,38.925000,2016,1,1,1002.581250,71.083333,145.589584
1,alabama,jefferson,2016-01-02 00:00:00.000000,1,73,38.339584,2016,1,2,1001.658334,67.083334,124.310417
2,alabama,jefferson,2016-01-03 00:00:00.000000,1,73,40.785416,2016,1,3,996.637500,67.020834,120.292708
3,alabama,jefferson,2016-01-04 00:00:00.000000,1,73,36.147917,2016,1,4,1000.566667,62.520834,84.986458
4,alabama,jefferson,2016-01-05 00:00:00.000000,1,73,37.666667,2016,1,5,1006.175000,41.687500,30.745833


In [14]:
print(len(data_t_ap_h_w_2016))
print(len(data_t_ap_h_w_2016)/365)
data_t_ap_h_w_2016.to_csv("./cleaned_data/EPA_weather_2016.csv", index=None, header=True)

144244
395.1890410958904
